분석순서 : dic -> concat -> corr (변수선정)

### HWP파일 -> 파이썬으로 불러오기 (8차 기본DB)

In [2]:
import olefile
import zlib
import struct

In [1]:
def get_hwp_text(filename):
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
       ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section"+str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed:
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data
    
        # 각 Section 내 text 추출    
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67] and header != 69207107 and header != 18875459 and  header != 35652675 :
                rec_data = unpacked_data[i+4:i+4+rec_len]
                #print(header)
                #print(rec_data.decode('UTF-16'))
                section_text += rec_data.decode('UTF-16')
                section_text += "\n"

            i += 4 + rec_len
        
        text += section_text
        text += "\n"

    return text

한자에 해당하는 header값 : 69207107
18875459
35652675
29363267

In [3]:
decoded_ns8 = get_hwp_text('C:/sas_view/ns_8.hwp')
decoded_ns8 

'mod_d\r\n최종 DB 수정일\r\nID\r\n개인 아이디\r\nID_fam\r\n가구 아이디\r\nyear\r\n조사연도\r\nregion\r\n17개 시도\r\ntown_t\r\n동/읍면 구분\r\napt_t\r\n아파트 구분\r\npsu\r\n조사구번호\r\nsex\r\n성별\r\nage\r\n만나이\r\nage_month\r\n월령\r\nincm\r\n소득 4분위수(개인)\r\nho_incm\r\n소득 4분위수(가구)\r\nincm5\r\n소득 5분위수(개인)\r\nho_incm5\r\n소득 5분위수(가구)\r\nedu\r\n교육수준 재분류 코드\r\noccp\r\n직업재분류 및 실업/비경제활동상태 코드\r\nwt_hs\r\n가구조사 가중치\r\nwt_itvex\r\n건강설문-검진조사 가중치\r\nwt_oe\r\n구강검사 가중치\r\nwt_pft\r\n폐기능검사 가중치(만40세이상)\r\nwt_ex1\r\n이비인후검사 가중치(만40세이상)\r\nwt_ige\r\n알레르겐검사 가중치(만10세이상)\r\nwt_ntr\r\n영양조사 가중치\r\nwt_tot\r\n건강설문-검진조사 & 영양조사 가중치\r\nwt_oe\r\n구강검사 & 영양조사 가중치\r\nwt_pfnt\r\n폐기능검사 & 영양조사 가중치(만40세이상)\r\nwt_ex1nt\r\n이비인후검사 & 영양조사 가중치(만40세이상)\r\nwt_igent\r\n알레르겐검사 & 영양조사 가중치(만10세이상)\r\nkstrata\r\n분산추정 층 \r\ncfam\r\n가구원수\r\ngenertn\r\n가구 세대구성코드\r\nallownc\r\n기초생활수급 여부\r\nhouse\r\n주택소유여부\r\nlive_t\r\n주택형태\r\nainc_unit1\r\n가구총소득(개방형): 소득단위\r\nainc_1\r\n가구총소득(개방형): 소득액\r\nainc\r\n월평균 가구총소득\r\nmarri_1\r\n결혼여부\r\nmarri_2\r\n결혼상태\r\nfam_rela\r\n가구주와의 관계\r\ntins\r\n

In [4]:
var_ns8 = decoded_ns8.split('\r\n')
var_ns8

['mod_d',
 '최종 DB 수정일',
 'ID',
 '개인 아이디',
 'ID_fam',
 '가구 아이디',
 'year',
 '조사연도',
 'region',
 '17개 시도',
 'town_t',
 '동/읍면 구분',
 'apt_t',
 '아파트 구분',
 'psu',
 '조사구번호',
 'sex',
 '성별',
 'age',
 '만나이',
 'age_month',
 '월령',
 'incm',
 '소득 4분위수(개인)',
 'ho_incm',
 '소득 4분위수(가구)',
 'incm5',
 '소득 5분위수(개인)',
 'ho_incm5',
 '소득 5분위수(가구)',
 'edu',
 '교육수준 재분류 코드',
 'occp',
 '직업재분류 및 실업/비경제활동상태 코드',
 'wt_hs',
 '가구조사 가중치',
 'wt_itvex',
 '건강설문-검진조사 가중치',
 'wt_oe',
 '구강검사 가중치',
 'wt_pft',
 '폐기능검사 가중치(만40세이상)',
 'wt_ex1',
 '이비인후검사 가중치(만40세이상)',
 'wt_ige',
 '알레르겐검사 가중치(만10세이상)',
 'wt_ntr',
 '영양조사 가중치',
 'wt_tot',
 '건강설문-검진조사 & 영양조사 가중치',
 'wt_oe',
 '구강검사 & 영양조사 가중치',
 'wt_pfnt',
 '폐기능검사 & 영양조사 가중치(만40세이상)',
 'wt_ex1nt',
 '이비인후검사 & 영양조사 가중치(만40세이상)',
 'wt_igent',
 '알레르겐검사 & 영양조사 가중치(만10세이상)',
 'kstrata',
 '분산추정 층 ',
 'cfam',
 '가구원수',
 'genertn',
 '가구 세대구성코드',
 'allownc',
 '기초생활수급 여부',
 'house',
 '주택소유여부',
 'live_t',
 '주택형태',
 'ainc_unit1',
 '가구총소득(개방형): 소득단위',
 'ainc_1',
 '가구총소득(개방형): 소득액',
 'ainc',
 '월평균 가구총

In [5]:
## 딕셔너리 형태로 
ns8_d = dict()
for i in range(0,len(var_ns8)-1,2):
    ns8_d[var_ns8[i]] = var_ns8[i+1] 

print(ns8_d)   

{'mod_d': '최종 DB 수정일', 'ID': '개인 아이디', 'ID_fam': '가구 아이디', 'year': '조사연도', 'region': '17개 시도', 'town_t': '동/읍면 구분', 'apt_t': '아파트 구분', 'psu': '조사구번호', 'sex': '성별', 'age': '만나이', 'age_month': '월령', 'incm': '소득 4분위수(개인)', 'ho_incm': '소득 4분위수(가구)', 'incm5': '소득 5분위수(개인)', 'ho_incm5': '소득 5분위수(가구)', 'edu': '교육수준 재분류 코드', 'occp': '직업재분류 및 실업/비경제활동상태 코드', 'wt_hs': '가구조사 가중치', 'wt_itvex': '건강설문-검진조사 가중치', 'wt_oe': '구강검사 & 영양조사 가중치', 'wt_pft': '폐기능검사 가중치(만40세이상)', 'wt_ex1': '이비인후검사 가중치(만40세이상)', 'wt_ige': '알레르겐검사 가중치(만10세이상)', 'wt_ntr': '영양조사 가중치', 'wt_tot': '건강설문-검진조사 & 영양조사 가중치', 'wt_pfnt': '폐기능검사 & 영양조사 가중치(만40세이상)', 'wt_ex1nt': '이비인후검사 & 영양조사 가중치(만40세이상)', 'wt_igent': '알레르겐검사 & 영양조사 가중치(만10세이상)', 'kstrata': '분산추정 층 ', 'cfam': '가구원수', 'genertn': '가구 세대구성코드', 'allownc': '기초생활수급 여부', 'house': '주택소유여부', 'live_t': '주택형태', 'ainc_unit1': '가구총소득(개방형): 소득단위', 'ainc_1': '가구총소득(개방형): 소득액', 'ainc': '월평균 가구총소득', 'marri_1': '결혼여부', 'marri_2': '결혼상태', 'fam_rela': '가구주와의 관계', 'tins': '건강보험종류', 'npins': '민

#### 딕셔너리 (ns8_d) -> 데이터프레임 변환

In [6]:
import pandas as pd

In [8]:
ns8_df = pd.DataFrame(list(ns8_d.items()),
                   columns = ['변수명','변수설명'])

ns8_df

,변수명,변수설명
0,mod_d,최종 DB 수정일
1,ID,개인 아이디
2,ID_fam,가구 아이디
3,year,조사연도
4,region,17개 시도
...,...,...
765,Y_SUP_YN,영양제 복용 여부
766,Y_SUP_KD1,비타민/무기질제
767,Y_SUP_KD3,유산균영양제/정장제
768,Y_SUP_KD4,초유영양제


In [10]:
## 데이터 프레임 (ns8_df) -> csv로 내보내기
ns8_df.to_csv('ns8_code.csv',encoding='euc-kr')